# Analysing 10 years of meditation data from Meditation Helper plus app

They don't make it easy!

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from datetime import datetime



# Import data and convert to dataframe

In [2]:
#read in data as text
with open('exported-sittings-20231005092003.csv') as f:
    rawdata = f.readlines()

#convert to ascii with list comprehension
asciidata = [line.encode('ascii', 'replace') for line in rawdata]



In [3]:
sittings = pd.DataFrame(columns = ['StartTime', 'EndTime', 'SessionType', 'Comment','Duration'])

#parse data
for row in asciidata:
    row = str(row).split('?')
    if len(row) > 2:
        start = datetime.strptime(row[2], '%Y-%m-%d %H:%M:%S')
        end = datetime.strptime(row[3], '%Y-%m-%d %H:%M:%S')
    if len(row) == 5:
        sittings.loc[len(sittings)] = {'StartTime': start, 'EndTime': end, 'SessionType': int(row[4])/1000}
    elif len(row) == 6:
        sittings.loc[len(sittings)] = {'StartTime': start, 'EndTime': end, 'SessionType': int(row[4])/1000, 'Comment': row[5]}

In [19]:
sittings.head()

,StartTime,EndTime,SessionType,Comment,Duration
0,2013-08-01 23:10:00,2013-08-01 23:20:00,600.0,\n',600
1,2013-08-02 21:03:30,2013-08-02 21:10:30,360.0,\n',420
2,2013-08-03 14:41:08,2013-08-03 14:48:08,360.0,\n',420
3,2013-08-04 20:58:38,2013-08-04 21:05:38,360.0,\n',420
4,2013-08-04 21:05:47,2013-08-04 21:12:47,360.0,\n',420


In [17]:
def toseconds(x):
    return x.seconds

In [18]:
sittings["Duration"] = (sittings["EndTime"] - sittings["StartTime"])
sittings["Duration"] = sittings["Duration"].apply(toseconds())


In [20]:
sittings.to_csv('sittings.csv', index=False)

In [49]:
def activityInPeriod(sits, starttime, endtime ):
    '''Find all sittings between two times and return a [PeriodLengthInMinutes x 1] array
    with one cell per minute containing Zero if not sitting, one if sitting.
    Includes partial sittings (starting before and ending after the period)
    For each interval we round seconds to nearest minute (up or down). 
    '''

    periodLength =   (endtime - starttime).total_seconds()
    thisPeriod = np.zeros(int(periodLength/60), dtype=int)
    #all sittings that start or end in this period
    theseSittings = sits[(sits.StartTime.between(starttime, endtime)).bool()]
    #theseSittings = sits[(sits.StartTime.between(starttime, endtime) or sits.EndTime.between(starttime , endtime))]
    for s in theseSittings:
        #find the start and end minutes of this sitting
        startminute = max((s.StartTime - starttime).seconds/60,0)
        endminute = min((s.EndTime - starttime).seconds/60, (endtime - starttime).seconds/60)
        thisPeriod[startminute:endminute] = 1

    return thisPeriod

    

def activityToday(sittingdata, todaysDate,daystart = "02:30"):
    '''Find all sittings today and return a 1440 x 1 array 
    with one cell per s containing Zero if not sitting, one if sitting
    '''

In [51]:
sits = sittings
starttime = datetime(2023,10,4,0,0,0)
endtime = datetime(2023,10,5,0,0,0)

theseSittings = sits[(sits.StartTime.between(starttime, endtime)).bool()]


ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().